# CLASSIFICADOR DE IMAGENS COM EUROSAT
Streamlit: https://modelo-eurosat.streamlit.app/

Imagens de Teste: https://drive.google.com/drive/folders/1siz7JmkvruHbtROUL1R420ce7ivD-NBq?usp=sharing

https://github.com/Mazalem/Modelo-EuroSAT

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
# 1. Carregar o dataset EuroSAT
(ds_train, ds_test), ds_info = tfds.load(
    'eurosat',
    split=['train[:80%]', 'train[80%:]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

CLASS_NAMES = ds_info.features['label'].names
IMG_SHAPE = ds_info.features['image'].shape
NUM_CLASSES = ds_info.features['label'].num_classes

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/eurosat/rgb/incomplete.WHHBOT_2.0.0/eurosat-train.tfrecord*...:   0%|     …

Dataset eurosat downloaded and prepared to /root/tensorflow_datasets/eurosat/rgb/2.0.0. Subsequent calls will reuse this data.


In [ ]:
# 2. Pré-processamento
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

BATCH_SIZE = 64

train_ds = ds_train.map(preprocess).cache().shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = ds_test.map(preprocess).batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)

In [ ]:
# 3. Data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomTranslation(0.1, 0.1)
])

In [ ]:
# 4. Modelo CNN
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=IMG_SHAPE),
    data_augmentation,

    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(32, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
    tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
    tf.keras.layers.GlobalAveragePooling2D(),

    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# 5. Treinamento
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=50,
    callbacks=[early_stop, reduce_lr]
)


Epoch 1/50
338/338 ━━━━━━━━━━━━━━━━━━━━ 25s 53ms/step - accuracy: 0.2404 - loss: 1.9478 - val_accuracy: 0.5439 - val_loss: 1.2628 - learning_rate: 0.0010
Epoch 2/50
338/338 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - accuracy: 0.5440 - loss: 1.2458 - val_accuracy: 0.6354 - val_loss: 0.9835 - learning_rate: 0.0010
Epoch 3/50
338/338 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.6422 - loss: 0.9858 - val_accuracy: 0.6859 - val_loss: 0.8076 - learning_rate: 0.0010
Epoch 4/50
338/338 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.6964 - loss: 0.8229 - val_accuracy: 0.7594 - val_loss: 0.6521 - learning_rate: 0.0010
Epoch 5/50
338/338 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - accuracy: 0.7353 - loss: 0.7248 - val_accuracy: 0.7254 - val_loss: 0.7463 - learning_rate: 0.0010
Epoch 6/50
338/338 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.7530 - loss: 0.6738 - val_accuracy: 0.7541 - val_loss: 0.6763 - learning_rate: 0.0010
Epoch 7/50
338/338 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 0.7720 - loss: 

In [ ]:
# 6. Avaliação final
loss, acc = model.evaluate(test_ds)
print(f"✅ Acurácia final no EuroSAT: {acc * 100:.2f}%")

85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9259 - loss: 0.2140
✅ Acurácia final no EuroSAT: 92.57%


In [ ]:
# 7. Salvar o modelo treinado
model.save("modelo_eurosat.keras")

In [ ]:
# 8. Exportar para TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("modelo_eurosat.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ Modelo salvo como .keras e .tflite com sucesso.")

Saved artifact at '/tmp/tmphchwko58'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name='keras_tensor_134')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  138839687536976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138839687538512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138839687537936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138839687539280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138839687538704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138839687540048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138839687450256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138839689721424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138839689720848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138839689721808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13883968972

In [ ]:
import tensorflow_datasets as tfds
from PIL import Image
import os

# Lista de classes
CLASS_NAMES = [
    'AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial',
    'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake'
]

# Carrega o dataset
ds_test = tfds.load('eurosat', split='train[80%:]', as_supervised=True)

# Cria uma pasta
os.makedirs('imagens_eurosat', exist_ok=True)

# Gera 10 imagens
for i, (img, label) in enumerate(ds_test.take(10)):
    arr = img.numpy()
    classe = CLASS_NAMES[label.numpy()]
    Image.fromarray(arr).save(f"imagens_eurosat/{i}_{classe}.png")

print("✅ Imagens salvas na pasta `imagens_eurosat/`")


✅ Imagens salvas na pasta `imagens_eurosat/`
